In [2]:
# Date: 18.01.25
# -- Part 1
# Note: Langchain 'ConversationalRetrievalChain' (aka RAG) with Gradio UI


# Date: 18.01.25
# # -- Part 2
# Note: Experimenting with langchain agents
# First version of this crashed and all was lost, so this is round 2
# Source: https://python.langchain.com/docs/tutorials/agents/

# Date: 26.01.25
# -- Part 3
# Note: Langchain experiments (1. conversational memory, )

#### Part 1:
Copy of 'week5, Community contributions, day 5 - ollama_rag_1' but using Flash'. <br>This shows Flash can be called in the way required for Langchanin agents. See Part 2

In [1]:
print('hello')

hello


In [2]:
print('hello')
import os
import io
import sys
from dotenv import load_dotenv
#from openai import OpenAI
import google.generativeai
#import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

import os
import google.generativeai as genai
genai.configure(api_key= api_key)

hello
There might be a problem with your API key? Please visit the troubleshooting notebook!


In [3]:
#pip install langchain-google-genai

In [4]:
# From week5, Community contributions, day 5 - ollama_rag_1

# -- 1.0 Imports
import os
import glob
from dotenv import load_dotenv
import gradio as gr
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
#from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# imports for langchain
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter


# -- 2.0 Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob("knowledge-base/*")

# With thanks to CG and Jon R, students on the course, for this fix needed for some users 
text_loader_kwargs = {'encoding': 'utf-8'}
# If that doesn't work, some Windows users might need to uncomment the next line instead
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)
len(documents)


# -- 3.0 Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk
# Chroma is a popular open source Vector Database based on SQLLite
DB_NAME = "vector_db"

#embeddings = OllamaEmbeddings(model="nomic-embed-text")
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Delete if already exists
if os.path.exists(DB_NAME):
    Chroma(persist_directory=DB_NAME, embedding_function=embeddings).delete_collection()

# Create vectorstore
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=DB_NAME)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")



C:\Users\Tim_S\AppData\Local\Temp\ipykernel_15036\2874214652.py:52: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
Created a chunk of size 1088, which is longer than the specified 1000


Vectorstore created with 123 documents


In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_retries=2
)

In [7]:
# create a new Chat with Ollama (well now Gem, but was Ollama in week5 version)
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
#MODEL = "llama3.2:latest" # This was in the code
#MODEL = llm # replacing above with above cell
#llm = ChatOllama(temperature=0.7, model=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

C:\Users\Tim_S\AppData\Local\Temp\ipykernel_15036\3098265202.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [ ]:
# Let's try a simple question

#query = "How do I change the water bottle ?"
query = "Who is avery?"
result = conversation_chain.invoke({"question": query})
print(result["answer"])

In [22]:
# Wrapping that in a function

def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [ ]:
# And in Gradio:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

#### Part 2:
How to use the prebuilt ReAct agent

In [10]:
# https://github.com/langchain-ai/langchain link to 
# https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/

In [12]:
%%capture --no-stderr
%pip install -U langgraph

In [13]:
# First we initialize the model we want to use.
#from langchain_openai import ChatOpenAI
#model = ChatOpenAI(model="gpt-4o", temperature=0)
model = llm

# For this tutorial we will use custom tool that returns pre-defined values for weather in two cities (NYC & SF)

from typing import Literal
from langchain_core.tools import tool


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]


# Define the graph
from langgraph.prebuilt import create_react_agent

graph = create_react_agent(model, tools=tools)

Usage<br>
First, let's visualize the graph we just created

In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

In [15]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

Let's run the app with an input that needs a tool call

In [ ]:
inputs = {"messages": [("user", "what is the weather in sf")]}
print_stream(graph.stream(inputs, stream_mode="values"))

Now let's try a question that doesn't need tools

In [ ]:
inputs = {"messages": [("user", "who built you?")]}
print_stream(graph.stream(inputs, stream_mode="values"))

Unrelated, but checking if it has memory

In [ ]:
inputs = {"messages": [("user", "How do i make pumpkin pie?")]}
#inputs = {"messages": [("user", "What about Cherry?")]}
print_stream(graph.stream(inputs, stream_mode="values"))

#### Part 3:
Langchain experiments

##### 1. chat memory

In [ ]:
# not sure if i need anything else from part one to make this code independant from above
print('hello')
import os
import io
import sys
from dotenv import load_dotenv
#from openai import OpenAI
import google.generativeai
#import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

import os
import google.generativeai as genai
genai.configure(api_key= api_key)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_retries=2
)

In [ ]:
# Below works, but no memory
# Source: https://python.langchain.com/docs/tutorials/chatbot/
from langchain_core.messages import HumanMessage
out = llm.invoke([HumanMessage(content="How much wood, can a woodchuck chuck if a woodchuck could chuck wood?")])
print(out.content)

In [12]:
# Trying to add memory persistence (same url)
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

model = llm

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
# Config: enables us to support multiple conversation threads with a single application
config = {"configurable": {"thread_id": "abc123"}}
#query = "Hi! I'm Bob."
#query = "Do i share a first name with any famous australian prime ministers?."
#query = "Is my name an anagram?."
query="Is my name short for anything?"


input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Bob is often short for Robert.


In [2]:
# Unrelated test of something...

In [5]:
def my_function(x):
    return x * 2

# # Assign the function to a variable
function_variable = my_function

# # Now you can call the function using the variable:
result = function_variable(5)  # Exactly the same as my_function(5)
print(result)  # Output: 10

# # You can even reassign the variable:
# def another_function(x):
#     return x + 10

# function_variable = another_function  # Now function_variable refers to a different function

# result = function_variable(5)  # Now calls another_function(5)
# print(result)  # Output: 15

10


In [8]:
class Example:
    def methoda(self):
        print("This is a method aaa.")

    def methodb(self):
        print("This is a method bbb.")

def function():
    print("This is a function.")

# Calling the function
#function()  # ✅ Works directly

# Creating an instance of Example
# ex = Example()
# ex.methoda()  # ✅ Called on an instance
Example.methodb(ex)  # ✅ Equivalent to ex.method()

This is a method bbb.
